In [1]:
import os
import sys
import json
import argparse
import numpy as np
import math
from einops import rearrange
import time
import random
import string
import h5py
from tqdm import tqdm
import gc

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
from accelerate import Accelerator

from statistics import *

from torchmetrics.regression import PearsonCorrCoef

os.chdir("/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/src")

# SDXL unCLIP requires code from https://github.com/Stability-AI/generative-models/tree/main
sys.path.append('generative_models/')
import sgm
from generative_models.sgm.modules.encoders.modules import FrozenOpenCLIPImageEmbedder # bigG embedder

# tf32 data type is faster than standard float32
torch.backends.cuda.matmul.allow_tf32 = True

# custom functions #
from utils import *

In [ ]:
def test(holdout_subject=1, top_n_rois=-1, data_path="../dataset/"):
    
    # Initialize an empty list to store the dataset names
    dataset_names = []

    with h5py.File(f'{data_path}/kastner_rois.hdf5', 'r') as file:
        # Function to recursively collect dataset names
        def collect_names(name, obj):
            if isinstance(obj, h5py.Dataset):
                dataset_names.append(name)

        # Iterate through the file structure and collect dataset names
        file.visititems(collect_names)
        
    
    
    
    
    with h5py.File(f'{data_path}/kastner_rois.hdf5', 'r') as file:
        roi = f['betas'][:]
        betas = torch.from_numpy(betas).to("cpu")
        
        beta_file = f"{data_path}/preprocessed_data/subject{subject}/whole_brain_include_heldout.pt"
        x = torch.load(beta_file).requires_grad_(False).to("cpu")
        
    # Function to recursively print the structure of the file
    def print_structure(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Group: {name}")
        elif isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}, shape: {obj.shape}, dtype: {obj.dtype}")

    # Iterate through the file structure
    file.visititems(print_structure)
    
    # betas = file['betas'][:]
    # betas = torch.from_numpy(betas).to("cpu")

In [ ]:
test(holdout_subject=1, top_n_rois=-1, data_path="/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset")

In [3]:
with h5py.File('/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset/kastner_rois.hdf5', 'r') as file:
    # Function to recursively print the structure of the file
    def print_structure(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Group: {name}")
        elif isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}, shape: {obj.shape}, dtype: {obj.dtype}")

    # Iterate through the file structure
    file.visititems(print_structure)
    
    # betas = file['betas'][:]
    # betas = torch.from_numpy(betas).to("cpu")

Group: subj01
Dataset: subj01/FEF, shape: (238508,), dtype: bool
Dataset: subj01/IPS0, shape: (238508,), dtype: bool
Dataset: subj01/IPS1, shape: (238508,), dtype: bool
Dataset: subj01/IPS2, shape: (238508,), dtype: bool
Dataset: subj01/IPS3, shape: (238508,), dtype: bool
Dataset: subj01/IPS4, shape: (238508,), dtype: bool
Dataset: subj01/IPS5, shape: (238508,), dtype: bool
Dataset: subj01/LO1, shape: (238508,), dtype: bool
Dataset: subj01/LO2, shape: (238508,), dtype: bool
Dataset: subj01/PHC1, shape: (238508,), dtype: bool
Dataset: subj01/PHC2, shape: (238508,), dtype: bool
Dataset: subj01/SPL1, shape: (238508,), dtype: bool
Dataset: subj01/TO1, shape: (238508,), dtype: bool
Dataset: subj01/TO2, shape: (238508,), dtype: bool
Dataset: subj01/V1d, shape: (238508,), dtype: bool
Dataset: subj01/V1v, shape: (238508,), dtype: bool
Dataset: subj01/V2d, shape: (238508,), dtype: bool
Dataset: subj01/V2v, shape: (238508,), dtype: bool
Dataset: subj01/V3A, shape: (238508,), dtype: bool
Dataset:

In [11]:

with h5py.File('/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset/kastner_rois.hdf5', 'r') as file:
    for roi in file['subj01'].keys():
        print(roi)
        for subject in file.keys():
            print(subject)
            mask = file[subject][roi]

FEF
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
IPS0
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
IPS1
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
IPS2
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
IPS3
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
IPS4
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
IPS5
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
LO1
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
LO2
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
PHC1
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
PHC2
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
SPL1
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
TO1
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
TO2
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
V1d
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
V1v
subj01
subj02
subj03
subj04
subj05
subj06
subj07
subj08
V2d
subj01
subj02
subj03
subj04

In [12]:
brain_region_masks = {}
with h5py.File('/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset/kastner_rois.hdf5', "r") as file:
    # Iterate over each subject
    for subject in file.keys():
        subject_group = file[subject]
        subject_masks = {}
        # Load the masks data for each subject
        for region in subject_group.keys():
            subject_masks[region] = subject_group[region][:]
        brain_region_masks[subject] = subject_masks
        
subject_masks = brain_region_masks[f"subj01"]

In [2]:
# Compute Pearson correlation along the 18 trials (axis 0) for each of the 72 elements
def pearson_corr(x, y):
    # Mean of each row (across trials)
    mean_x = torch.mean(x, dim=0)
    mean_y = torch.mean(y, dim=0)
    
    # Covariance numerator
    cov = torch.sum((x - mean_x) * (y - mean_y), dim=0)
    
    # Standard deviations
    std_x = torch.sqrt(torch.sum((x - mean_x) ** 2, dim=0))
    std_y = torch.sqrt(torch.sum((y - mean_y) ** 2, dim=0))
    
    # Pearson correlation
    corr = cov / (std_x * std_y)
    
    return corr

In [3]:
brain_region_masks = {}
pearson = PearsonCorrCoef(num_outputs=18)

with h5py.File('/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset/roi_collection.hdf5', "r") as file:
    
    # Iterate over each subject
    for subject in file.keys():
        subject_group = file[subject]
        subject_masks = {}
        # Load the masks data for each subject
        for region in subject_group.keys():
            subject_masks[region] = subject_group[region][:]
        brain_region_masks[subject] = subject_masks
    
    for i in tqdm(range(1, 9), desc="Calculating Subject Rank Order ROIs"):
        
        # Create the list of subjects
        subjects = [f'subj0{j}' for j in range(1, 9)]
        
        # Remove the current subject without reassigning to 'subjects'
        subjects.remove(f'subj0{i}')
        
        print(f"   All subjects used for rank order calculation: {subjects}")

        # Initialize an empty dictionary to store ROI correlations
        roi_correlations = {}

        # Loop through each ROI for correlation analysis, with tqdm progress bar
        for roi in tqdm(file[f'subj0{i}'].keys(), desc="Processing ROIs"):
            
            roi_data = []  # To store ROI data across all subjects
            print(f"\nProcessing ROI: {roi}")
                
            # Load the ROI data for each subject
            for subject in subjects:
                print(f"  Loading subject: {subject}")
                    
                # Load the beta file for the subject
                beta_vision, _ = load_nsd_mental_imagery(subject=subject[-1:], mode='vision', stimtype="all", average=True, nest=False, whole_brain=True)
                beta_imagery, _ = load_nsd_mental_imagery(subject=subject[-1:], mode='imagery', stimtype="all", average=True, nest=False, whole_brain=True)
                
                print(f"    Shape of beta_vision: {beta_vision.shape}")
                print(f"    Shape of beta_imagery: {beta_imagery.shape}")
                
                # Load the boolean mask for the current ROI
                roi_mask = file[subject][roi]
                # nsd_general = np.array(file[subject]['nsd_general'])
                # print(type(nsd_general))
                # print(nsd_general)
                # print(f"  Loaded ROI mask for {subject}{roi}, mask shape: {roi_mask.shape}, True count: {np.sum(roi_mask)}")
                # print(f"  Loaded ROI mask for {subject}{roi}, mask shape: {nsd_general.shape}, True count: {np.sum(nsd_general)}")
                # roi_mask = roi_mask[nsd_general]
                print(f"  Loaded ROI mask for {subject}{roi}, mask shape: {roi_mask.shape}, True count: {np.sum(roi_mask)}")
                
                # Apply the ROI mask to the subject data (along the second dimension)
                masked_data_vision = beta_vision[..., roi_mask]
                masked_data_imagery = beta_imagery[..., roi_mask]
                print(f"    Shape of masked data vision: {masked_data_vision.shape}")
                print(f"    Shape of masked data imagery: {masked_data_imagery.shape}")
                
                # Remove the singleton dimension (1) to reduce the shape.
                masked_data_vision = masked_data_vision.squeeze(1)
                masked_data_imagery = masked_data_imagery.squeeze(1)
                
                # Compute the Pearson correlation for the two tensors
                #correlation = pearson_corr(masked_data_vision, masked_data_imagery)

                # Append the single pearson correlation value to 
                # the subject correlation tensor to the list
                # ---- Samplewise -----
                subject_specific_correlation_value = torch.mean(pearson(masked_data_vision.moveaxis(0, 1), masked_data_imagery.moveaxis(0, 1)))
                # ---- Voxelwise -----
                # pearson = PearsonCorrCoef(num_outputs=masked_data_vision.shape[-1])
                # subject_specific_correlation_value = torch.mean(pearson(masked_data_vision, masked_data_imagery))
                print(f"    Subject specific correlation value: {subject_specific_correlation_value}")
                
                roi_data.append(subject_specific_correlation_value)
                
            # Stack the tensors along a new dimension and calculate the mean
            mean_roi_pearson_correlation_across_subjects = torch.mean(torch.stack(roi_data), dim=0)
            print(f"    ROI: {roi} Mean Pearson Correlation Across Subjects: {mean_roi_pearson_correlation_across_subjects}")
            
            # Store the mean correlation for the ROI
            # .item() converts the torch object to a float value.
            roi_correlations[roi] = mean_roi_pearson_correlation_across_subjects.item()
                    
        # Sort ROIs by mean Pearson correlation
        sorted_rois = sorted(roi_correlations.items(), key=lambda x: x[1], reverse=True)

        # Print the ROIs ranked by correlation
        print("\nROIs ranked by Pearson correlation across subjects:")
        for roi, corr in sorted_rois:
            print(f"ROI: {roi}, Mean Pearson Correlation: {corr:.4f}")
            
        # Convert the sorted list back into a dictionary
        sorted_rois_dict = {roi: corr for roi, corr in sorted_rois}

        # Save the sorted ROIs and their correlations to a JSON file
        with open(f"/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset/subj0{i}_sorted_rois_rank_order.json", "w") as json_file:
            json.dump(sorted_rois_dict, json_file, indent=4)

Calculating Subject Rank Order ROIs:   0%|          | 0/8 [00:00<?, ?it/s]

   All subjects used for rank order calculation: ['subj02', 'subj03', 'subj04', 'subj05', 'subj06', 'subj07', 'subj08']



Processing ROI: 35
  Loading subject: subj02
torch.Size([18, 1, 242606]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 242606]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision: torch.Size([18, 1, 242606])
    Shape of beta_imagery: torch.Size([18, 1, 242606])
  Loaded ROI mask for subj0235, mask shape: (242606,), True count: 204


/tmp/ipykernel_33068/906299071.py:56: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  masked_data_vision = beta_vision[..., roi_mask]
/tmp/ipykernel_33068/906299071.py:57: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  masked_data_imagery = beta_imagery[..., roi_mask]


    Shape of masked data vision: torch.Size([18, 1, 204])
    Shape of masked data imagery: torch.Size([18, 1, 204])
    Subject specific correlation value: 0.09128984063863754
  Loading subject: subj03
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision: torch.Size([18, 1, 246730])
    Shape of beta_imagery: torch.Size([18, 1, 246730])
  Loaded ROI mask for subj0335, mask shape: (246730,), True count: 204
    Shape of masked data vision: torch.Size([18, 1, 204])
    Shape of masked data imagery: torch.Size([18, 1, 204])
    Subject specific correlation value: 0.053883831948041916
  Loading subject: subj04
torch.Size([18, 1, 229642]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 229642]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision: torch.Size([18, 1, 229642])
    Shape of beta_imagery: torch.Size([18, 1, 229642])
  Loaded ROI mask for subj0435, mask shape: (229642,), True count: 248
 

    Shape of masked data vision: torch.Size([18, 1, 227])
    Shape of masked data imagery: torch.Size([18, 1, 227])
    Subject specific correlation value: 0.039323508739471436
    ROI: 35 Mean Pearson Correlation Across Subjects: 0.04212961718440056

Processing ROI: 36
  Loading subject: subj02
torch.Size([18, 1, 242606]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 242606]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision: torch.Size([18, 1, 242606])
    Shape of beta_imagery: torch.Size([18, 1, 242606])
  Loaded ROI mask for subj0236, mask shape: (242606,), True count: 780
    Shape of masked data vision: torch.Size([18, 1, 780])
    Shape of masked data imagery: torch.Size([18, 1, 780])
    Subject specific correlation value: 0.07398863136768341
  Loading subject: subj03
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision: torch.Size([18, 1, 246730])
    Shape of beta_imagery: torch.

Calculating Subject Rank Order ROIs:   0%|          | 0/8 [06:38<?, ?it/s]


KeyboardInterrupt: 

In [13]:
print(subject_masks)

{'FEF': array([False, False, False, ..., False, False, False]), 'IPS0': array([False, False, False, ..., False, False, False]), 'IPS1': array([False, False, False, ..., False, False, False]), 'IPS2': array([False, False, False, ..., False, False, False]), 'IPS3': array([False, False, False, ..., False, False, False]), 'IPS4': array([False, False, False, ..., False, False, False]), 'IPS5': array([False, False, False, ..., False, False, False]), 'LO1': array([False, False, False, ..., False, False, False]), 'LO2': array([False, False, False, ..., False, False, False]), 'PHC1': array([False, False, False, ..., False, False, False]), 'PHC2': array([False, False, False, ..., False, False, False]), 'SPL1': array([False, False, False, ..., False, False, False]), 'TO1': array([False, False, False, ..., False, False, False]), 'TO2': array([False, False, False, ..., False, False, False]), 'V1d': array([False, False, False, ..., False, False, False]), 'V1v': array([False, False, False, ..., False

In [2]:
import nibabel as nib
import numpy as np

def load_subject_masks(subject_ids, data_path):
    """
    Load the combined masks and label files for all subjects and store them in a dictionary.
    
    Args:
        subject_ids (list): List of subject identifiers (e.g., ['subj01', 'subj02', ..., 'subj08']).
        data_path (str): Base directory path for the dataset.
    
    Returns:
        dict: A dictionary containing masks and labels for each subject.
    """
    subject_masks = {}

    # Iterate over all subject IDs
    for subject_id in subject_ids:
        
        # Load the combined mask for the subject
        # ---- NSD General + ROIs -----
        mask_path = f"{data_path}/combined_masks/{subject_id}_combined_mask_nsd_general.nii.gz"
        # ---- Just ROIs -----
        # mask_path = f"{data_path}/combined_masks/{subject_id}_combined_mask.nii.gz"
        mask = nib.load(mask_path).get_fdata()

        # Load the brainmask_inflated for the subject
        brainmask_path = f"{data_path}/nsddata/ppdata/{subject_id}/func1pt8mm/roi/brainmask_inflated_1.0.nii"
        brainmask_inflated = nib.load(brainmask_path).get_fdata()

        # Clean up brainmask_inflated and apply it to the mask
        brainmask_inflated = np.nan_to_num(brainmask_inflated)
        brainmask_inflated = np.where(brainmask_inflated == 1.0, True, False)
        
        mask = mask[brainmask_inflated]

        # Load the labels from the corresponding label file
        # ---- NSD General + ROIs -----
        labels_path = f"{data_path}/combined_masks/{subject_id}_labels_nsd_general.txt"
        # ---- Just ROIs -----
        # labels_path = f"{data_path}/combined_masks/{subject_id}_labels.txt"
        label_to_roi = {}
        with open(labels_path, 'r') as label_file:
            for line in label_file:
                idx, roi_name = line.strip().split(": ")
                if roi_name != 'No Mask':
                    label_to_roi[int(idx)] = roi_name
                    
        # Apply brainmask to the mask
        filtered_mask = {}
        for number, roi in label_to_roi.items():
            filtered_mask[roi] = mask == number

        # Store the filtered mask and label mapping for the current subject
        subject_masks[subject_id] = {
            "filtered_mask": filtered_mask,
            "label_to_roi": label_to_roi
        }

    return subject_masks

# Example usage:
subject_ids = [f'subj0{i}' for i in range(1, 9)]  # ['subj01', 'subj02', ..., 'subj08']
data_path = "/home/naxos2-raid25/kneel027/home/kneel027/MindEye_Imagery/dataset"
subject_masks = load_subject_masks(subject_ids, data_path)

In [10]:
from concurrent.futures import ThreadPoolExecutor
import torch.multiprocessing as mp
import nibabel as nib
import numpy as np

def load_subject_masks(subject_ids, data_path):
    subject_masks = {}
    all_rois = set()

    # Preload masks for all subjects
    for subject_id in subject_ids:
        print(f"Loading combined mask for {subject_id}...")
        # ---- NSD General + ROIs -----
        mask_path = f"{data_path}/combined_masks/{subject_id}_combined_mask_nsd_general.nii.gz"
        # ---- Just ROIs -----
        # mask_path = f"{data_path}/combined_masks/{subject_id}_combined_mask.nii.gz"
        mask = nib.load(mask_path).get_fdata()

        brainmask_path = f"{data_path}/nsddata/ppdata/{subject_id}/func1pt8mm/roi/brainmask_inflated_1.0.nii"
        brainmask_inflated = nib.load(brainmask_path).get_fdata()

        # Clean up brainmask
        brainmask_inflated = np.nan_to_num(brainmask_inflated)
        brainmask_inflated = np.where(brainmask_inflated == 1.0, True, False)

        print(f"Applying brainmask to combined mask for {subject_id}...")
        mask = mask[brainmask_inflated]

        # Load ROI labels
        print(f"Loading ROI labels for {subject_id}...")
        # ---- NSD General + ROIs -----
        labels_path = f"{data_path}/combined_masks/{subject_id}_labels_nsd_general.txt"
        # ---- Just ROIs -----
        # labels_path = f"{data_path}/combined_masks/{subject_id}_labels.txt"
        label_to_roi = {}
        with open(labels_path, 'r') as label_file:
            for line in label_file:
                idx, roi_name = line.strip().split(": ")
                if roi_name != 'No Mask':
                    label_to_roi[int(idx)] = roi_name
                    all_rois.add(roi_name)

        # Create filtered masks for ROIs
        filtered_mask = {roi: mask == number for number, roi in label_to_roi.items()}
        subject_masks[subject_id] = {"filtered_mask": filtered_mask, "label_to_roi": label_to_roi}

        print(f"Completed loading and processing for {subject_id}.\n")
        
    # Find ROIs common to all subjects
    shared_rois = all_rois.intersection(*[set(subject_masks[subj]["filtered_mask"].keys()) for subj in subject_ids])
    print(f"ROIs shared by all subjects: {shared_rois}")

    return subject_masks, shared_rois

# Parallelized correlation calculation
def process_roi(roi, subjects, pearson, subject_masks):
    print(f"\nProcessing ROI: {roi}")
    roi_data = []

    for subj in subjects:
        print(f"  Loading beta data for {subj}...")
        
        # if roi not in subject_masks[subj]['filtered_mask']:
        #     continue

        # Load the beta vision and imagery data
        beta_vision, _ = load_nsd_mental_imagery(subject=subj[-1:], mode='vision', stimtype="all", average=True, nest=False, whole_brain=True)
        beta_imagery, _ = load_nsd_mental_imagery(subject=subj[-1:], mode='imagery', stimtype="all", average=True, nest=False, whole_brain=True)

        # Print shapes of the beta tensors
        print(f"    Shape of beta_vision for {subj}: {beta_vision.shape}")
        print(f"    Shape of beta_imagery for {subj}: {beta_imagery.shape}")
        
        # Load and apply the ROI mask
        print(f"  Applying mask for ROI {roi} to subject {subj} data...")
        roi_mask = subject_masks[subj]['filtered_mask'][roi]

        # Print the shape of the ROI mask
        print(f"    Shape of ROI mask for {subj}, ROI {roi}: {roi_mask.shape}")
        print(f"    Number of active voxels in ROI {roi}: {np.sum(roi_mask)}")

        # Apply the mask to the beta data
        masked_data_vision = beta_vision[..., roi_mask]
        masked_data_imagery = beta_imagery[..., roi_mask]

        # Print shapes of masked data
        print(f"    Shape of masked_data_vision for {subj}, ROI {roi}: {masked_data_vision.shape}")
        print(f"    Shape of masked_data_imagery for {subj}, ROI {roi}: {masked_data_imagery.shape}")

        # Squeeze singleton dimensions
        masked_data_vision = masked_data_vision.squeeze(1)
        masked_data_imagery = masked_data_imagery.squeeze(1)

        # Compute the Pearson correlation for the two tensors
        print(f"    Computing Pearson correlation for subject {subj}, ROI {roi}...")
        
        # ---- Voxelwise -----
        # pearson = PearsonCorrCoef(num_outputs=masked_data_vision.shape[-1])
        # subject_specific_correlation_value = torch.mean(pearson(masked_data_vision, masked_data_imagery))
        
        # ---- Samplewise -----
        subject_specific_correlation_value = torch.mean(pearson(masked_data_vision.moveaxis(0, 1), masked_data_imagery.moveaxis(0, 1)))

        # Store the correlation result
        print(f"    Pearson correlation for subject {subj}, ROI {roi}: {subject_specific_correlation_value}")
        roi_data.append(subject_specific_correlation_value)

    # Compute the mean correlation across all subjects for this ROI
    print(f"    ROI Data: {roi_data}")
    mean_roi_correlation = torch.mean(torch.stack(roi_data), dim=0)

    # Print the mean correlation value for this ROI across subjects
    print(f"Mean Pearson correlation across subjects for ROI {roi}: {mean_roi_correlation}\n")
    return roi, mean_roi_correlation.item()


# Example usage
subject_ids = [f'subj0{i}' for i in range(1, 9)]
data_path = "/home/naxos2-raid25/kneel027/home/kneel027/MindEye_Imagery/dataset"
subject_masks, shared_rois = load_subject_masks(subject_ids, data_path)

subject_list = [3, 4, 6, 8]
for i in tqdm(subject_list, desc="Calculating Subject Rank Order ROIs"):
    print(f"\nStarting calculations for subject subj0{i}...")
    subjects = [f'subj0{j}' for j in subject_list if j != i]
    # subjects = [f'subj0{j}' for j in subject_list]
    print(f" Subjects being used: {subjects}")
    roi_correlations = {}

    # ---- Samplewise -----
    pearson = PearsonCorrCoef(num_outputs=18)

    # Parallel processing for each ROI
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_roi, roi, subjects, pearson, subject_masks) for roi in shared_rois]
        
        for future in tqdm(futures, desc="Processing ROIs"):
            roi, correlation = future.result()
            roi_correlations[roi] = correlation

    # Sort and save results
    sorted_rois = sorted(roi_correlations.items(), key=lambda x: x[1], reverse=True)
    print(f"\nROIs ranked by Pearson correlation for subj0{i}:")
    for roi, corr in sorted_rois:
        print(f"  ROI: {roi}, Mean Pearson Correlation: {corr:.4f}")

    with open(f"/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset/all_sorted_rois_rank_order_samplewise_nsd_general.json", "w") as json_file:
        json.dump({roi: corr for roi, corr in sorted_rois}, json_file, indent=4)

    print(f"Finished processing for subject subj0{i}.\n")


Loading combined mask for subj01...
Applying brainmask to combined mask for subj01...
Loading ROI labels for subj01...
Completed loading and processing for subj01.

Loading combined mask for subj02...
Applying brainmask to combined mask for subj02...
Loading ROI labels for subj02...
Completed loading and processing for subj02.

Loading combined mask for subj03...
Applying brainmask to combined mask for subj03...
Loading ROI labels for subj03...
Completed loading and processing for subj03.

Loading combined mask for subj04...
Applying brainmask to combined mask for subj04...
Loading ROI labels for subj04...
Completed loading and processing for subj04.

Loading combined mask for subj05...
Applying brainmask to combined mask for subj05...
Loading ROI labels for subj05...
Completed loading and processing for subj05.

Loading combined mask for subj06...
Applying brainmask to combined mask for subj06...
Loading ROI labels for subj06...
Completed loading and processing for subj06.

Loading co

Calculating Subject Rank Order ROIs:   0%|          | 0/1 [00:00<?, ?it/s]


Starting calculations for subject subj01...
 Subjects being used: ['subj03', 'subj04', 'subj06', 'subj08']

Processing ROI: CA2
  Loading beta data for subj03...

Processing ROI: 36
  Loading beta data for subj03...

Processing ROI: SPL1
  Loading beta data for subj03...

Processing ROI: PPA
  Loading beta data for subj03...

Processing ROI: dorsomedialPul
  Loading beta data for subj03...

Processing ROI: 35
  Loading beta data for subj03...

Processing ROI: lateral
  Loading beta data for subj03...

Processing ROI: DG
  Loading beta data for subj03...

Processing ROI: FBA-2
  Loading beta data for subj03...

Processing ROI: V1
  Loading beta data for subj03...

Processing ROI: V3A
  Loading beta data for subj03...

Processing ROI: IPS5
  Loading beta data for subj03...

Processing ROI: ventral
  Loading beta data for subj03...

Processing ROI: V2
  Loading beta data for subj03...

Processing ROI: RSC
  Loading beta data for subj03...

Processing ROI: IPS3
  Loading beta data for sub

torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 246730]) torch.Size([18, 3, 425, 425])
torch.Size([18

torch.Size([18, 1, 234961]) torch.Size([18, 3, 425, 425])
torch.Size([18, 1, 234961]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision for subj08: torch.Size([18, 1, 234961])
    Shape of beta_imagery for subj08: torch.Size([18, 1, 234961])
  Applying mask for ROI CA2 to subject subj08 data...
    Shape of ROI mask for subj08, ROI CA2: (234961,)
    Number of active voxels in ROI CA2: 19
    Shape of masked_data_vision for subj08, ROI CA2: torch.Size([18, 1, 19])
    Shape of masked_data_imagery for subj08, ROI CA2: torch.Size([18, 1, 19])
    Computing Pearson correlation for subject subj08, ROI CA2...
    Pearson correlation for subject subj08, ROI CA2: -0.049877531826496124
    ROI Data: [tensor(0.0261), tensor(0.0416), tensor(0.0193), tensor(-0.0499)]
Mean Pearson correlation across subjects for ROI CA2: 0.009282903745770454

    Shape of beta_vision for subj08: torch.Size([18, 1, 234961])
    Shape of beta_imagery for subj08: torch.Size([18, 1, 234961])
  Applying mask for R

...

torch.Size([18, 1, 234961]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision for subj08: torch.Size([18, 1, 234961])
    Shape of beta_imagery for subj08: torch.Size([18, 1, 234961])
  Applying mask for ROI SC to subject subj08 data...
    Shape of ROI mask for subj08, ROI SC: (234961,)
    Number of active voxels in ROI SC: 13
    Shape of masked_data_vision for subj08, ROI SC: torch.Size([18, 1, 13])
    Shape of masked_data_imagery for subj08, ROI SC: torch.Size([18, 1, 13])
    Computing Pearson correlation for subject subj08, ROI SC...
    Pearson correlation for subject subj08, ROI SC: -0.0009656001930125058
    ROI Data: [tensor(0.0381), tensor(0.1149), tensor(-0.0161), tensor(-0.0010)]
Mean Pearson correlation across subjects for ROI SC: 0.03397674858570099

torch.Size([18, 1, 234961]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision for subj08: torch.Size([18, 1, 234961])
    Shape of beta_imagery for subj08: torch.Size([18, 1, 234961])
  Applying mask for ROI IPS1

Processing ROIs: 100%|██████████| 36/36 [01:13<00:00,  2.05s/it]

torch.Size([18, 1, 234961]) torch.Size([18, 3, 425, 425])
    Shape of beta_vision for subj08: torch.Size([18, 1, 234961])
    Shape of beta_imagery for subj08: torch.Size([18, 1, 234961])
  Applying mask for ROI CA1 to subject subj08 data...
    Shape of ROI mask for subj08, ROI CA1: (234961,)
    Number of active voxels in ROI CA1: 239
    Shape of masked_data_vision for subj08, ROI CA1: torch.Size([18, 1, 239])
    Shape of masked_data_imagery for subj08, ROI CA1: torch.Size([18, 1, 239])
    Computing Pearson correlation for subject subj08, ROI CA1...
    Pearson correlation for subject subj08, ROI CA1: 0.052240949124097824
    ROI Data: [tensor(-0.0232), tensor(0.0061), tensor(0.0130), tensor(0.0522)]
Mean Pearson correlation across subjects for ROI CA1: 0.012046406976878643




Calculating Subject Rank Order ROIs: 100%|██████████| 1/1 [01:14<00:00, 74.07s/it]


ROIs ranked by Pearson correlation for subj01:
  ROI: ERC, Mean Pearson Correlation: 0.1433
  ROI: dorsolateralPul, Mean Pearson Correlation: 0.0835
  ROI: IPS3, Mean Pearson Correlation: 0.0591
  ROI: nsdgeneral, Mean Pearson Correlation: 0.0557
  ROI: IPS2, Mean Pearson Correlation: 0.0552
  ROI: IPS1, Mean Pearson Correlation: 0.0533
  ROI: PPA, Mean Pearson Correlation: 0.0511
  ROI: V3, Mean Pearson Correlation: 0.0474
  ROI: SPL1, Mean Pearson Correlation: 0.0472
  ROI: IPS0, Mean Pearson Correlation: 0.0471
  ROI: lateral, Mean Pearson Correlation: 0.0395
  ROI: EBA, Mean Pearson Correlation: 0.0390
  ROI: IPS5, Mean Pearson Correlation: 0.0375
  ROI: RSC, Mean Pearson Correlation: 0.0357
  ROI: SC, Mean Pearson Correlation: 0.0340
  ROI: V3A, Mean Pearson Correlation: 0.0338
  ROI: FBA-2, Mean Pearson Correlation: 0.0333
  ROI: parietal, Mean Pearson Correlation: 0.0285
  ROI: V1, Mean Pearson Correlation: 0.0268
  ROI: FEF, Mean Pearson Correlation: 0.0245
  ROI: PHC, Mean Pe